# GeneViking - prokkaViking protoype

In [7]:
from Bio import Entrez
from Bio import SeqIO
import io
import pandas as pd
import re
import os
import subprocess


__author__ = 'Natalia Quinones-Olvera'
__email__ = "nquinones@g.harvard.edu"
Entrez.email = __email__

Location parser

In [ ]:
def parse_loc(loc, ref_start):
    'Function to correctly parse location object'
    # start
    if str(type(loc.start)) == "<class 'Bio.SeqFeature.ExactPosition'>":
        start = loc.start + ref_start
        start_str = start
    elif str(type(loc.start)) == "<class 'Bio.SeqFeature.BeforePosition'>":
        start = loc.start + ref_start
        start_str = '<' + str(start)
    elif str(type(loc.start)) == "<class 'Bio.SeqFeature.AfterPosition'>":
        start = loc.start + ref_start
        start_str = '>' + str(start)
    else:
        start_str = 'unknown position type'

    # end
    if str(type(loc.end)) == "<class 'Bio.SeqFeature.ExactPosition'>":
        end = loc.end + ref_start - 1
        end_str = end
    elif str(type(loc.end)) == "<class 'Bio.SeqFeature.BeforePosition'>":
        end = loc.end + ref_start - 1 
        end_str = '<' + str(end)
    elif str(type(loc.end)) == "<class 'Bio.SeqFeature.AfterPosition'>":
        end = loc.end + ref_start -1
        end_str = '>' + str(end)
    else:
        end_str = 'unknown position type'

    if loc.strand == 1:
        strand = '+'
    else:
        strand = '-'

    return [start, end, start_str, end_str, strand]

Protein coords

In [ ]:
def get_prot_coords(protein_acc):
    '''
    Helper function to find protein coordinates from accession.
    Has limited functionality
    '''
    # check if it is non-redundant
    if protein_acc.find('WP') == 0:
        print('Warning: Seems that you\'re providing a non-redundant protein accession.')

    # send request to NCBI protein database
    epost = Entrez.epost('protein', id=protein_acc)
    request = Entrez.read(epost)

    response = Entrez.efetch(db='protein',
                             webenv=request['WebEnv'],
                             query_key=request['QueryKey'],
                             rettype="gb",
                             retmode="text")

    response_io = io.StringIO(response.read())

    feat_list = []

    for gb_record in SeqIO.parse(response_io, "genbank"):
        for feat in gb_record.features:
            feat_list.append(feat.type)
                # source feature
            if feat.type == 'CDS':
                coded_str = feat.qualifiers['coded_by'][0].split(':')
                acc = coded_str[0]
                coords = coded_str[1].split('..')
                start = coords[0]
                end = coords[1]

    # if no CDS feature found, return error message
    if 'CDS' not in feat_list:
        print('Error: Couldn\'t find CDS feature. Unable to find coordinates.')
        return None

    return acc, (start, end)

GeneViking

In [ ]:
def gene_viking(acc, start, end, thresh, output):
    '''
    Main functionality.
    '''
    start_2 = start - thresh
    end_2 = end + thresh

    if start_2 < 0:
        start_2 = 0

    epost = Entrez.epost('nuccore', id=acc)
    request = Entrez.read(epost)
    response = Entrez.efetch(db='nuccore',
                             webenv=request['WebEnv'],
                             query_key=request['QueryKey'],
                             rettype="gb",
                             retmode="text",
                             seq_start=start_2,
                             seq_stop=end_2)

    response_io = io.StringIO(response.read())

    query_range = set(range(start, end))

    all_l = []

    for record in SeqIO.parse(response_io, 'genbank'):
        for feat in record.features:
            if feat.type == 'CDS':
                if 'pseudo' not in feat.qualifiers:
                    fstart, fend, fstart_str, fend_str, fstrand = parse_loc(feat.location, start_2)
                    prot_id = feat.qualifiers['protein_id'][0]
                    product = feat.qualifiers['product'][0]

                    feat_range = set(range(fstart, fend))

                    if len(query_range.intersection(feat_range)) == 0:
                        status = ''
                    else:
                        status = '!'

                    feats = [prot_id, product, fstart_str, fend_str, fstrand, status]

                    all_l.append(feats)

    df = pd.DataFrame(all_l)

    df.columns = ['prot_acc', 'product', 'start', 'end', 'strand', 'query_overlap']

    if output is not None:
        df.to_csv(output, sep='\t', index=None)

    return df

## 1. Single request download

In [ ]:
acc_test01 = 'NZ_CP025054.1'
start_test01 = 12910
end_test01 = 13287
thresh_test01 = 10000
output = None
download = True

### 1.1. Rewrite modules

In [1]:
class NCBIQuery:
    
    def __init__(self, acc, start, end, thresh):
        
        # ncbi accession
        self.acc = acc
        
        # query range
        self.query_range = set(range(start, end))
        # query start
        self.start = start
        # query end
        self.end = end
        
        # neighborhood start
        self.rangestart = start - thresh
        if self.rangestart < 0:
            self.rangestart = 0
        # neighborhood end
        self.rangeend = end + thresh
        
    def parse_loc(self, loc, ref_start):
        '''
        Function to correctly parse location object
        '''
        
        # start
        if str(type(loc.start)) == "<class 'Bio.SeqFeature.ExactPosition'>":
            start = loc.start + ref_start
            start_str = start
        elif str(type(loc.start)) == "<class 'Bio.SeqFeature.BeforePosition'>":
            start = loc.start + ref_start
            start_str = '<' + str(start)
        elif str(type(loc.start)) == "<class 'Bio.SeqFeature.AfterPosition'>":
            start = loc.start + ref_start
            start_str = '>' + str(start)
        else:
            start_str = 'unknown position type'

        # end
        if str(type(loc.end)) == "<class 'Bio.SeqFeature.ExactPosition'>":
            end = loc.end + ref_start - 1
            end_str = end
        elif str(type(loc.end)) == "<class 'Bio.SeqFeature.BeforePosition'>":
            end = loc.end + ref_start - 1 
            end_str = '<' + str(end)
        elif str(type(loc.end)) == "<class 'Bio.SeqFeature.AfterPosition'>":
            end = loc.end + ref_start -1
            end_str = '>' + str(end)
        else:
            end_str = 'unknown position type'

        if loc.strand == 1:
            strand = '+'
        else:
            strand = '-'

        return [start, end, start_str, end_str, strand]

    
    def get_gb(self):
        '''
        Requests genbank file with Entrez
        '''
        
        epost = Entrez.epost('nuccore', id=self.acc)
        request = Entrez.read(epost)

        response = Entrez.efetch(db='nuccore',
                                 webenv=request['WebEnv'],
                                 query_key=request['QueryKey'],
                                 rettype='gb',
                                 retmode="text",
                                 seq_start=self.rangestart,
                                 seq_stop=self.rangeend)

        response_io = io.StringIO(response.read())

        return response_io
    
    
    
    def parse_gb(self, gbfile,  output=None):
        '''
        Parse gb file. Extracts product information from
        feature types: CDS, tRNA, rRNA, ncRNA, and repeat_region
        and writes dataframe with output.
        Can save table into tsv output
        '''

        feats = []

        for record in SeqIO.parse(gbfile, 'genbank'):
            for feat in record.features:
                # extract location object
                fstart, fend, fstart_str, fend_str, fstrand = self.parse_loc(feat.location, self.rangestart)
                # save range
                feat_range = set(range(fstart, fend))

                # check overlap
                if len(self.query_range.intersection(feat_range)) == 0:
                    status = ''
                else:
                    status = '!'


                if feat.type == 'CDS':
                    if 'pseudo' not in feat.qualifiers:
                        prot_id = feat.qualifiers['protein_id'][0]
                        product = feat.qualifiers['product'][0]

                        feat_info = [feat.type, prot_id, product, fstart_str, fend_str, fstrand, status]

                    else:
                        prot_id = 'None'
                        product = feat.qualifiers['product'][0]

                        feat_info = ['CDS_pseudo', prot_id, product, fstart_str, fend_str, fstrand, status]

                    feats.append(feat_info)

                if feat.type in ['tRNA', 'rRNA', 'ncRNA', 'repeat_region']:
                    product = feat.qualifiers['product'][0]

                    feat_info = [feat.type, None, product, fstart_str, fend_str, fstrand, status]

                    feats.append(feat_info)

        df = pd.DataFrame(feats)

        df.columns = ['type', 'acc', 'product', 'start', 'end', 'strand', 'query_overlap']

        if output is not None:
            df.to_csv(output, sep='\t', index=None)

        return df
    
    def gb_table(self, output=None):
        '''
        Parse gb file. Extracts product information from
        feature types: CDS, tRNA, rRNA, ncRNA, and repeat_region
        and writes dataframe with output.
        Can save table into tsv output
        '''
        
        response_gb = self.get_gb()
        
        df = self.parse_gb(response_gb, output=output)

        return df
    
    def get_fasta(self):
        '''
        Requests fasta file with Entrez
        '''
        epost = Entrez.epost('nuccore', id=self.acc)
        request = Entrez.read(epost)

        response = Entrez.efetch(db='nuccore',
                                 webenv=request['WebEnv'],
                                 query_key=request['QueryKey'],
                                 rettype='fasta',
                                 retmode="text",
                                 seq_start=self.rangestart,
                                 seq_stop=self.rangeend)

        response_io = io.StringIO(response.read())
        
        return response_io
    
    def parse_fasta(self, directory='.', file=None):
        '''
        Parses fasta file from Entrez request, saves
        into output
        '''
        response_fasta = self.get_fasta()
        
        for record in SeqIO.parse(response_fasta, 'fasta'):
            fastarecord = record
            
        # save output
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        if file is None:
            name = '{}.fa'.format(fastarecord.id.replace(':', '_'))
            output = os.path.join(directory, name)
        else:
            output = os.path.join(directory, file)
        
        SeqIO.write(fastarecord, output, 'fasta')
        
        return output
    
    def prokka_table(self, directory, output=None):
        '''
        '''
        file = self.parse_fasta(directory=directory)
        
        name = '{}_{}-{}'.format(self.acc, self.rangestart, self.rangeend)
        
        prokka_dir = os.path.join(directory, 'prokka')
        
        process = subprocess.run(['prokka',
                                  '--outdir', prokka_dir,
                                  '--prefix', name,
                                  '--compliant',
                                  file],
                                 stdout=subprocess.PIPE,
                                 stderr=subprocess.PIPE,
                                 universal_newlines=True)
        
        if process.returncode != 0:
            print('ERROR: {} \n {}'.format(process.returncode, process.stderr))
        else:
            print('Done. Saved in: {}'.format(prokka_dir))
            
            gbk_path = '{}/{}.gbk'.format(prokka_dir, name)
            
            df = self.parse_gb(gbk_path, output=output)
            
            return df
        

In [ ]:
class Viking:
    
    def __init__(self, entries, annot='ncbi'):
        self.annot = annot
        
        if self.annot in ['ncbi', 'prokka']:
            pass
        else:
            print('ERROR: Unrecognized annotation type.')
        
        self.entries = entries
        
        if not type(self.entries) == list:
            raise ValueError('entries should be a list of NCBIQuery objects')
        
        for entry in self.entries:
            if not isinstance(entry, NCBIQuery):
                raise ValueError('entries should be a list of NCBIQuery objects')

In [9]:
a = NCBIQuery('CP025053.1', 701514, 702401, 10000)

In [11]:
a.gb_table()

,type,acc,product,start,end,strand,query_overlap
0,CDS,AUA81440.1,ABC transporter permease,<691514,692465,+,
1,CDS,AUA81441.1,ABC transporter permease,692571,693401,+,
2,CDS,AUA81442.1,ribulose-phosphate 3-epimerase,693525,694199,+,
3,CDS,AUA81443.1,phosphoglycolate phosphatase,694199,695017,+,
4,CDS,AUA81444.1,anthranilate synthase component I,695090,696568,+,
5,CDS,AUA81445.1,transcription regulatory protein PrtN,696886,697200,-,
6,CDS,AUA81446.1,transcriptional regulator,697300,698070,-,
7,CDS,AUA81447.1,conjugal transfer protein TraR,698528,698728,+,
8,CDS,AUA81448.1,hypothetical protein,698776,699135,+,
9,CDS,AUA81449.1,holin,699498,699947,+,


In [13]:
a.prokka_table('tmp')

Done. Saved in: tmp/prokka


,type,acc,product,start,end,strand,query_overlap
0,CDS,Prokka:JBEMBNFH_00001,hypothetical protein,691551,692465,+,
1,CDS,Prokka:JBEMBNFH_00002,Inner membrane ABC transporter permease protei...,692571,693401,+,
2,CDS,Prokka:JBEMBNFH_00003,Ribulose-phosphate 3-epimerase,693525,694199,+,
3,CDS,Prokka:JBEMBNFH_00004,Phosphoglycolate phosphatase,694199,695017,+,
4,CDS,Prokka:JBEMBNFH_00005,Anthranilate synthase component 1,695090,696568,+,
5,CDS,Prokka:JBEMBNFH_00006,hypothetical protein,696886,697200,-,
6,CDS,Prokka:JBEMBNFH_00007,HTH-type transcriptional regulator PrtR,697300,698070,-,
7,CDS,Prokka:JBEMBNFH_00008,hypothetical protein,698155,698325,-,
8,CDS,Prokka:JBEMBNFH_00009,hypothetical protein,698528,698728,+,
9,CDS,Prokka:JBEMBNFH_00010,hypothetical protein,698776,699135,+,


In [14]:
! rm -r tmp

In [4]:
b = Viking([a])

NameError: name 'Viking' is not defined

In [ ]:
b.entries